# MDAnalysis vs micelle_whole tool

This Jupyter notebook aims to illustrate the improved algorithms developed in PySoftK to extend and complement the analysis tools available in codes like MDAnalysis. 

As an example, this is the system that we are going to try to reconstrut its connectivity:
![Image Alt Text](data/pictures_tutorial/snapshot_to_cluster_initial_mdanalysis_vs_pysoftk.png)

This is a complex system to reconnect, since it is divided along the three dimensions creating chunks of the system with different sizes. Let's first attempt to reconect it using MDAnalysis

## MDAnalysis transformations

In [3]:
import MDAnalysis as mda 
import MDAnalysis.analysis.distances
import numpy as np
import matplotlib.patches as mpatches
import itertools
import MDAnalysis.transformations as trans
from tqdm.auto import tqdm


/home/raquellrdc/Desktop/PhD/mda_umap/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's load the trajectory

In [5]:
topology='data/pictures_tutorial/triblock.tpr'
trajectory='data/pictures_tutorial/triblock.xtc'


u=mda.Universe(topology, trajectory)

The resid of the polymers that we want to cluster are the following, they are the ones displayed in the above image, obtained with the SCP tool

In [6]:
resids_to_be_clustered=[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


Now, let's run MDAnalysis transformations to attempt to make the micelle whole and center it in the box

In [16]:
u2=mda.Universe(topology, trajectory)


polymer=u.select_atoms('not resname SOL')

#converting the resids array into a list ot be passed onto the mdanalysis selecition of the atoms of the micelle
ind=[]


for item in resids_to_be_clustered:
        
        ind.append(str(item) +' ')
        
        
ind_f = ' '.join(ind)
    
    
    
micelle = polymer.select_atoms('resid ' + str(ind_f))
        
transforms = [trans.unwrap(micelle),
            trans.center_in_box(micelle, wrap=True)]
        
u2.trajectory.add_transformations(*transforms)
        
with MDAnalysis.Writer("data/pictures_tutorial/mdanalysis_whole.pdb", micelle.n_atoms) as W:
    
    W.write(micelle)

Let's visualize the mdanalysis_whole.pdb with VMD
![Image Alt Text](data/pictures_tutorial/mdanalysis_whole_screenshot.png)

Clearly, MDAnalysis has not been able to make the micelle whole, since its extension is greater than hald the box lengh.

# Pysoftk's make_micelle_whole 

Let's load the modules and trajectory

In [8]:
from  utils_mda import MDA_input
#from pysoftk.pol_analysis.tools.utils_mda import MDA_input
from utils_tools import *
#from pysoftk.pol_analysis.tools.utils_tools import *
from clustering import SCP
#from pysoftk.pol_analysis.clustering import SCP
from make_micelle_whole import micelle_whole
#from pysoftk.pol_analysis.make_micelle_whole import micelle_whole

import numpy as np
import pandas as pd

In [9]:
topology='data/pictures_tutorial/triblock.tpr'
trajectory='data/pictures_tutorial/triblock.xtc'


1. First, we need to run SCP to obtain the largest micelle resids

In [10]:
atom_names = ['C02X', 'C001']

cluster_cutoff = 12

results_name='data/pictures_tutorial/triblock_scp_result'

start=0
stop=1000
step=1


In [11]:
c = SCP(topology, trajectory).spatial_clustering_run(start, stop, step, atom_names, cluster_cutoff, results_name)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.45it/s]

The file data/pictures_tutorial/triblock_scp_result.parquet has been successfully created.
Function spatial_clustering_run Took 1.2031 seconds


2. Now, let's obtain the largest micelle resids

In [12]:
resids_total='data/pictures_tutorial/triblock_scp_result.parquet'

largest_micelle_resids = micelle_whole(topology, trajectory).obtain_largest_micelle_resids(resids_total)

3. Let's make the micelle whole

In [13]:
resname=['LIG']

start=0
stop=10001
step=1

atom_pos = micelle_whole(topology, trajectory).running_make_cluster_whole(resname, largest_micelle_resids, start, stop, step)

  0%|                                                                                                                                                    | 0/1 [00:00<?, ?it/s]/home/raquellrdc/Desktop/PhD/pysoftk/alejandro_newest_releast_check/pysoftk_analysis_code/test_final/make_micelle_whole.py:347: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  atom_positions_over_trajectory = list(tqdm(map(self.make_cluster_whole, frames, resname, cluster_resids_f[0],
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.79s/it]

Elapsed time for matrix calculation: 3.1445 seconds


4. Let's obtain the snapshot

In [14]:
snapshot_frame=0

snapshot_name='data/pictures_tutorial/pysoftk_whole.pdb'

atom_pos_frame=atom_pos[0]

largest_micelle_resids_frame=largest_micelle_resids[0]


Now, atom_pos contains the coordinates of all the atoms of the micelle made whole at each time step selected. In each array, the first element is the time frame of the analysis, and the second the positions array

In [15]:
snapshot = micelle_whole(topology, trajectory).obtain_snapshot(snapshot_name, atom_pos_frame, 
                                                               largest_micelle_resids_frame, resname, snapshot_frame)

6534
6534


/home/raquellrdc/Desktop/PhD/mda_umap/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/raquellrdc/Desktop/PhD/mda_umap/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/raquellrdc/Desktop/PhD/mda_umap/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/home/raquellrdc/Desktop/PhD/mda_umap/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  warnings.warn("Found no information for attr: '{}'"
/home/raquellrdc/Desktop/PhD/mda_umap/lib/p

This is how the micelle looks now:

![Image Alt Text](data/pictures_tutorial/pysoftk_whole_screenshot.png
)


Clearly, PySoftK is able to make the micelle whole even when its length is greater than half the box size!